In [121]:
%pip install --user pandas
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\USUARIO\Desktop\quiniela origen.xlsx")

In [122]:
import scipy
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import jaccard_score
from sklearn.neighbors import KNeighborsClassifier

In [123]:
df.drop(['Temporada','Local', 'Visitante', 'V','V.1','E','E.1','D','D.1', 'sin perder', 'sin ganar','sin perder.1', 'sin ganar.1' ], axis=1, inplace=True)

In [124]:
df= df.dropna(subset=['resultado'], axis=0)

In [125]:
df.drop(['Jornada', 'dif.', 'dif..1'], axis=1, inplace=True)

In [126]:
df['Puntos Local']=pd.cut(df['PUNTOS'], bins=3, labels=['menos de 3', 'de 3 a 5', '6 o más'], right=False)
df['Puntos Visitante']=pd.cut(df['PUNTOS.1'], bins=3, labels=['menos de 3', 'de 3 a 5', '6 o más'], right=False)

In [127]:
df['MPSP_l']=pd.qcut(df['partidos consecutivos sin perder'],q= [0, 0.4, 1])
df['MPSG_l']=pd.qcut(df['media de partidos sin ganar'], q= [0, 0.4, 1])
df['MPSP_v']=pd.qcut(df['partidos consecutivos sin perder.1'], q= [0, 0.4, 1])
df['MPSG_v']=pd.qcut(df['media de partidos sin ganar.1'], q= [0, 0.4, 1])

In [128]:
#df = df[df['cuartil']!=1]
#df.reset_index(drop=True)

In [129]:
df['GF_local'] = pd.qcut(df['AF'], q= [0, 0.6, 1], labels = ['0-1.33', '>1.33'])
df['GC_local'] = pd.qcut(df['AC'], q= [0, 0.55, 1], labels=['0-1.2', '>1.2'])
#este es un elemento clave, decidir los intervalos de los goles a favor y en contra de local y visitante

In [130]:
df['GF_visitante'] = pd.qcut(df['AF.1'], q= [0, 0.6, 1], labels = ['0-1.1', '>1.1'])
df['GC_visitante'] = pd.qcut(df['AC.1'], q= [0, 0.55, 1], labels = ['0-1.1', '>1.4'])
df

,cuartil,partidos consecutivos sin perder,media de partidos sin ganar,PUNTOS,último partido como local,último partido,AF,AC,cuartil.1,partidos consecutivos sin perder.1,...,Puntos Local,Puntos Visitante,MPSP_l,MPSG_l,MPSP_v,MPSG_v,GF_local,GC_local,GF_visitante,GC_visitante
0,1,6.800000,1.333333,9,3,3,2.380000,0.250000,2,4.142857,...,6 o más,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,0-1.2,>1.1,0-1.1
1,1,6.800000,1.300000,6,0,3,1.480000,1.190000,2,1.666667,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",0-1.33,>1.2,0-1.1,>1.4
2,1,4.428571,1.555556,7,1,1,2.420000,1.080000,4,2.000000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,0-1.2,0-1.1,0-1.1
3,3,2.400000,2.250000,4,3,3,1.380000,0.630000,1,4.428571,...,de 3 a 5,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,0-1.2,>1.1,0-1.1
4,3,2.555556,2.900000,4,0,0,1.640000,1.560000,1,6.800000,...,de 3 a 5,6 o más,"(2.27, 12.0]","(2.25, 11.0]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,>1.2,>1.1,0-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3,2.000000,2.000000,4,3,1,1.333333,1.000000,4,1.500000,...,de 3 a 5,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,0-1.2,0-1.1,>1.4
778,1,12.000000,0.000000,9,3,3,2.600000,1.000000,4,1.500000,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,0-1.2,0-1.1,>1.4
779,2,2.250000,2.000000,1,0,0,1.333333,0.666667,2,2.000000,...,menos de 3,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,0-1.2,0-1.1,0-1.1
780,2,2.666667,1.666667,6,3,0,1.200000,1.200000,2,2.250000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,>1.2,0-1.1,0-1.1


In [131]:
df['resultado'] = df['resultado'].replace(0, 2)

In [132]:
df['resultado'] = df['resultado'].replace(2, 0)

In [133]:
df

,cuartil,partidos consecutivos sin perder,media de partidos sin ganar,PUNTOS,último partido como local,último partido,AF,AC,cuartil.1,partidos consecutivos sin perder.1,...,Puntos Local,Puntos Visitante,MPSP_l,MPSG_l,MPSP_v,MPSG_v,GF_local,GC_local,GF_visitante,GC_visitante
0,1,6.800000,1.333333,9,3,3,2.380000,0.250000,2,4.142857,...,6 o más,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,0-1.2,>1.1,0-1.1
1,1,6.800000,1.300000,6,0,3,1.480000,1.190000,2,1.666667,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",0-1.33,>1.2,0-1.1,>1.4
2,1,4.428571,1.555556,7,1,1,2.420000,1.080000,4,2.000000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,0-1.2,0-1.1,0-1.1
3,3,2.400000,2.250000,4,3,3,1.380000,0.630000,1,4.428571,...,de 3 a 5,6 o más,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,0-1.2,>1.1,0-1.1
4,3,2.555556,2.900000,4,0,0,1.640000,1.560000,1,6.800000,...,de 3 a 5,6 o más,"(2.27, 12.0]","(2.25, 11.0]","(2.233, 12.0]","(-0.001, 2.25]",>1.33,>1.2,>1.1,0-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3,2.000000,2.000000,4,3,1,1.333333,1.000000,4,1.500000,...,de 3 a 5,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,0-1.2,0-1.1,>1.4
778,1,12.000000,0.000000,9,3,3,2.600000,1.000000,4,1.500000,...,6 o más,menos de 3,"(2.27, 12.0]","(-0.001, 2.25]","(0.999, 2.233]","(2.25, 11.0]",>1.33,0-1.2,0-1.1,>1.4
779,2,2.250000,2.000000,1,0,0,1.333333,0.666667,2,2.000000,...,menos de 3,6 o más,"(0.999, 2.27]","(-0.001, 2.25]","(0.999, 2.233]","(-0.001, 2.25]",0-1.33,0-1.2,0-1.1,0-1.1
780,2,2.666667,1.666667,6,3,0,1.200000,1.200000,2,2.250000,...,6 o más,de 3 a 5,"(2.27, 12.0]","(-0.001, 2.25]","(2.233, 12.0]","(-0.001, 2.25]",0-1.33,>1.2,0-1.1,0-1.1


# Histórico

## Dos años anteriores

---

In [134]:
chisq_cuartil = pd.crosstab(df['cuartil'], df['resultado'])
chisq_cuartil

resultado,0,1
cuartil,,
1,51,89
2,94,81
3,161,90
4,135,81


In [135]:
scipy.stats.contingency.chi2_contingency(chisq_cuartil)

(32.609186138125494,
 3.8937554782590154e-07,
 3,
 array([[ 78.95140665,  61.04859335],
        [ 98.68925831,  76.31074169],
        [141.54859335, 109.45140665],
        [121.81074169,  94.18925831]]))

---

In [136]:
chisq_cuartil_1 = pd.crosstab(df['cuartil.1'], df['resultado'])
chisq_cuartil_1

resultado,0,1
cuartil.1,,
1,112,26
2,94,80
3,147,106
4,88,129


In [137]:
scipy.stats.contingency.chi2_contingency(chisq_cuartil_1)

(57.26007892576182,
 2.261481758941785e-12,
 3,
 array([[ 77.82352941,  60.17647059],
        [ 98.12531969,  75.87468031],
        [142.67647059, 110.32352941],
        [122.37468031,  94.62531969]]))

---

# Racha

## Tres últimos partidos
---

In [138]:
chisq_PUNTOS_L = pd.crosstab(df['PUNTOS'], df['resultado'])
chisq_PUNTOS_L

resultado,0,1
PUNTOS,,
0,31,15
1,60,36
2,46,21
3,72,58
4,93,81
5,32,22
6,49,44
7,36,30
9,22,34


In [139]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_L)

(15.939331828476108,
 0.043256737248137646,
 8,
 array([[25.94117647, 20.05882353],
        [54.13810742, 41.86189258],
        [37.78388747, 29.21611253],
        [73.31202046, 56.68797954],
        [98.12531969, 75.87468031],
        [30.45268542, 23.54731458],
        [52.44629156, 40.55370844],
        [37.21994885, 28.78005115],
        [31.58056266, 24.41943734]]))

---

In [140]:
chisq_PUNTOS_Lo = pd.crosstab(df['Puntos Local'], df['resultado'])
chisq_PUNTOS_Lo

resultado,0,1
Puntos Local,,
menos de 3,137,72
de 3 a 5,197,161
6 o más,107,108


In [141]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_Lo)

(11.236089252080188,
 0.00363173557170762,
 2,
 array([[117.86317136,  91.13682864],
        [201.89002558, 156.10997442],
        [121.24680307,  93.75319693]]))

---

In [142]:
chisq_PUNTOS_V = pd.crosstab(df['PUNTOS.1'], df['resultado'])
chisq_PUNTOS_V

resultado,0,1
PUNTOS.1,,
0,13,22
1,51,45
2,36,35
3,69,51
4,81,63
5,32,25
6,63,46
7,66,43
9,30,11


In [143]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_V)
#p-value= 0.0002
#critical value = 29.7

(12.234831929014542,
 0.1410297897118424,
 8,
 array([[19.73785166, 15.26214834],
        [54.13810742, 41.86189258],
        [40.03964194, 30.96035806],
        [67.67263427, 52.32736573],
        [81.20716113, 62.79283887],
        [32.14450128, 24.85549872],
        [61.46930946, 47.53069054],
        [61.46930946, 47.53069054],
        [23.12148338, 17.87851662]]))

---

In [144]:
chisq_PUNTOS_Vi = pd.crosstab(df['Puntos Visitante'], df['resultado'])
chisq_PUNTOS_Vi

resultado,0,1
Puntos Visitante,,
menos de 3,100,102
de 3 a 5,182,139
6 o más,159,100


In [145]:
scipy.stats.contingency.chi2_contingency(chisq_PUNTOS_Vi)

(6.5392915866486065,
 0.038019891589833446,
 2,
 array([[113.91560102,  88.08439898],
        [181.02429668, 139.97570332],
        [146.0601023 , 112.9398977 ]]))

---

## Último partido
---

In [146]:
chisq_UPL = pd.crosstab(df['último partido como local'], df['resultado'])
chisq_UPL

resultado,0,1
último partido como local,,
0,132,95
1,142,79
3,167,167


In [147]:
scipy.stats.contingency.chi2_contingency(chisq_UPL)
#p-value= 0.03
#critical value = 7.0

(11.388622691653612,
 0.003365053739676434,
 2,
 array([[128.0140665 ,  98.9859335 ],
        [124.63043478,  96.36956522],
        [188.35549872, 145.64450128]]))

---

In [148]:
chisq_UP = pd.crosstab(df['último partido'], df['resultado'])
chisq_UP

resultado,0,1
último partido,,
0,183,137
1,119,88
3,139,116


In [149]:
scipy.stats.contingency.chi2_contingency(chisq_UP)

(0.5508051863082588,
 0.7592663862360295,
 2,
 array([[180.46035806, 139.53964194],
        [116.73529412,  90.26470588],
        [143.80434783, 111.19565217]]))

---

In [150]:
chisq_UPV = pd.crosstab(df['último partido.1'], df['resultado'])
chisq_UPV

resultado,0,1
último partido.1,,
0,138,109
1,134,93
3,169,139


In [151]:
scipy.stats.contingency.chi2_contingency(chisq_UPV)

(0.9602005439742862,
 0.618721348276499,
 2,
 array([[139.29283887, 107.70716113],
        [128.0140665 ,  98.9859335 ],
        [173.69309463, 134.30690537]]))

---

# Goles

## Local
---

In [152]:
chisq_gfl = pd.crosstab(df['GF_local'], df['resultado'])
chisq_gfl

resultado,0,1
GF_local,,
0-1.33,303,187
>1.33,138,154


In [153]:
scipy.stats.contingency.chi2_contingency(chisq_gfl)

(15.22153151134037,
 9.560710494792811e-05,
 1,
 array([[276.32992327, 213.67007673],
        [164.67007673, 127.32992327]]))

---

In [154]:
chisq_gcl = pd.crosstab(df['GC_local'], df['resultado'])
chisq_gcl

resultado,0,1
GC_local,,
0-1.2,227,203
>1.2,214,138


In [155]:
scipy.stats.contingency.chi2_contingency(chisq_gcl)

(4.723113349790621,
 0.02975981413122242,
 1,
 array([[242.49360614, 187.50639386],
        [198.50639386, 153.49360614]]))

---

## Visitante

---

In [156]:
chisq_gfv = pd.crosstab(df['GF_visitante'], df['resultado'])
chisq_gfv

resultado,0,1
GF_visitante,,
0-1.1,227,247
>1.1,214,94


In [157]:
scipy.stats.contingency.chi2_contingency(chisq_gfv)

(34.51557123221811,
 4.228550288846306e-09,
 1,
 array([[267.30690537, 206.69309463],
        [173.69309463, 134.30690537]]))

---

In [158]:
chisq_gcv = pd.crosstab(df['GC_visitante'], df['resultado'])
chisq_gcv

resultado,0,1
GC_visitante,,
0-1.1,262,170
>1.4,179,171


In [159]:
scipy.stats.contingency.chi2_contingency(chisq_gcv)

(6.722618477726181,
 0.009519775905817733,
 1,
 array([[243.62148338, 188.37851662],
        [197.37851662, 152.62148338]]))

---

## Desempeño del equipo

In [160]:
chisq_mpsp_l = pd.crosstab(df['MPSP_l'], df['resultado'])
chisq_mpsp_l

resultado,0,1
MPSP_l,,
"(0.999, 2.27]",204,109
"(2.27, 12.0]",237,232


In [161]:
scipy.stats.contingency.chi2_contingency(chisq_mpsp_l)

(15.777063036614969,
 7.126120888236549e-05,
 1,
 array([[176.51278772, 136.48721228],
        [264.48721228, 204.51278772]]))

---

In [162]:
chisq_mpsg_l = pd.crosstab(df['MPSG_l'], df['resultado'])
chisq_mpsg_l

resultado,0,1
MPSG_l,,
"(-0.001, 2.25]",130,190
"(2.25, 11.0]",311,151


In [163]:
scipy.stats.contingency.chi2_contingency(chisq_mpsg_l)

(53.6891367671245,
 2.348582821321265e-13,
 1,
 array([[180.46035806, 139.53964194],
        [260.53964194, 201.46035806]]))

---

In [164]:
chisq_mpsp_v = pd.crosstab(df['MPSP_v'], df['resultado'])
chisq_mpsp_v

resultado,0,1
MPSP_v,,
"(0.999, 2.233]",127,186
"(2.233, 12.0]",314,155


In [165]:
scipy.stats.contingency.chi2_contingency(chisq_mpsp_v)

(52.03899731907869,
 5.440924906224995e-13,
 1,
 array([[176.51278772, 136.48721228],
        [264.48721228, 204.51278772]]))

---

In [166]:
chisq_mpsg_v = pd.crosstab(df['MPSG_v'], df['resultado'])
chisq_mpsg_v

resultado,0,1
MPSG_v,,
"(-0.001, 2.25]",221,94
"(2.25, 11.0]",220,247


In [167]:
scipy.stats.contingency.chi2_contingency(chisq_mpsg_v)

(39.709161240086175,
 2.947396948170779e-10,
 1,
 array([[177.64066496, 137.35933504],
        [263.35933504, 203.64066496]]))

---